In [20]:
import openai
import json
import requests
from openai import OpenAI

import pandas as pd
import glob,os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import re

import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE = 142

In [21]:
def split_by_pages(filepath, encoding='utf-8'):
    """
    filepath : ocr 결과 txt 파일 경로 전달

    return : page별로 구분된 하나의 리스트를 반환
    """
    with open(filepath,'r',encoding=encoding) as f:
        data = f.readlines()
    text = []
    page_text = []
    for d in data:
        if re.compile(PAGE_PATTERN).match(d):
            if len(page_text)>0:
                text.append(' '.join(page_text))
            page_text = []
        page_text.append(d)
    text.append(''.join(page_text))
    return text

def erase_tag(text, tag):
    """
    text : split_by_page로 얻은 텍스트 리스트 또는 텍스트
    tag : 지우고 싶은 <tag>
    """
    tag_pattern = re.compile(f'<{tag}>|</{tag}>')
    if isinstance(text, list):
        text_ = [tag_pattern.sub('',x) for x in text]
        return text_
    else:
        text_ = tag_pattern.sub('',text)
        return text_

def extract_text_between_tag(text, tag):
    """
    text : split_by_page로 얻은 텍스트 리스트
    tag : <tag> 사이의 텍스트를 추출
    """
    # Create a regex pattern for the specified tag
    pattern = f'<{tag}>(.*?)</{tag}>'
    # Use re.findall to extract all occurrences between the specified tags
    matches = re.findall(pattern, text, re.DOTALL)
    return matches

In [26]:
class GPT():
    __classname__ = "OpenAI"
    api_key = ''
    client = None    
    def __init__(self, api_filepath):
        with open(api_filepath,'r') as f:
            ak = json.load(f)
        self.api_key = ak['OPENAI_API_KEY']
        self.client = OpenAI(api_key=self.api_key)
        self.EVALUATION_PROMPT_TEMPLATE = """
You will be given one summary written for an article. Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions very carefully. 
Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

{criteria}

Evaluation Steps:

{steps}

Example:

Source Text:

{document}

Summary:

{summary}

Evaluation Form (scores ONLY):

- {metric_name}
"""
        self.RELEVANCY_SCORE_CRITERIA = """
Relevance(1-5) - selection of important content from the source. \
The summary should include only important information from the source document. \
Annotators were instructed to penalize summaries which contained redundancies and excess information.
"""
        self.RELEVANCY_SCORE_STEPS = """
1. Read the summary and the source document carefully.
2. Compare the summary to the source document and identify the main points of the article.
3. Assess how well the summary covers the main points of the article, and how much irrelevant or redundant information it contains.
4. Assign a relevance score from 1 to 5.
"""
        self.COHERENCE_SCORE_CRITERIA = """
Coherence(1-5) - the collective quality of all sentences. \
We align this dimension with the DUC quality question of structure and coherence \
whereby "the summary should be well-structured and well-organized. \
The summary should not just be a heap of related information, but should build from sentence to a\
coherent body of information about a topic."
"""
        self.COHERENCE_SCORE_STEPS = """
1. Read the article carefully and identify the main topic and key points.
2. Read the summary and compare it to the article. Check if the summary covers the main topic and key points of the article,
and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.
"""
        self.CONSISTENCY_SCORE_CRITERIA = """
Consistency(1-5) - the factual alignment between the summary and the summarized source. \
A factually consistent summary contains only statements that are entailed by the source document. \
Annotators were also asked to penalize summaries that contained hallucinated facts.
"""
        self.CONSISTENCY_SCORE_STEPS = """
1. Read the article carefully and identify the main facts and details it presents.
2. Read the summary and compare it to the article. Check if the summary contains any factual errors that are not supported by the article.
3. Assign a score for consistency based on the Evaluation Criteria.
"""
        self.FLUENCY_SCORE_CRITERIA = """
Fluency(1-3): the quality of the summary in terms of grammar, spelling, punctuation, word choice, and sentence structure.
1: Poor. The summary has many errors that make it hard to understand or sound unnatural.
2: Fair. The summary has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.
3: Good. The summary has few or no errors and is easy to read and follow.
"""
        self.FLUENCY_SCORE_STEPS = """
Read the summary and evaluate its fluency based on the given criteria. Assign a fluency score from 1 to 3.
"""
        
    def get_chat_completion(self, msg, model='gpt-4o-mini', **kargs):
        response = self.client.chat.completions.create(
            model = model,
            messages = msg
        )
        return response.choices[0].message.content

    def get_embedding(self, sentence, model="text-embedding-3-small"):
       '''
       - pricing : text-embedding-3-small = $0.02/1M tokens
           텍스트가 많은 pdf는 대략 5,000 tokens -> pdf 200개에 0.02 달러(25~30원).
       text : 한 문장
       return : 한 문장에 대한 embedding (output dimension = 1536)
       '''
       return self.client.embeddings.create(input = sentence, model=model).data[0].embedding

    def get_geval_score(
        self, document: str, summary: str, model: str = 'gpt-4o-mini'
    ):
        '''
        document : 원본 문서
        summary : 요약 텍스트

        return : 요약 텍스트에 대한 relevance, coherence, consistency, fluency G-EVAL 점수
        '''
        evaluation_metrics = {
            "Relevance": (RELEVANCY_SCORE_CRITERIA, RELEVANCY_SCORE_STEPS),
            "Coherence": (COHERENCE_SCORE_CRITERIA, COHERENCE_SCORE_STEPS),
            "Consistency": (CONSISTENCY_SCORE_CRITERIA, CONSISTENCY_SCORE_STEPS),
            "Fluency": (FLUENCY_SCORE_CRITERIA, FLUENCY_SCORE_STEPS)
        }
        scores = []
        for evaluation_type, (criteria, steps) in evaluation_metrics.items():
            prompt = self.EVALUATION_PROMPT_TEMPLATE.format(
                criteria=criteria,
                steps=steps,
                metric_name=evaluation_type,
                document=document,
                summary=summary
            )
            response = self.client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                max_tokens=5,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
            )
            # 논문에서는 GPT-4의 parameter를 다음과 같이 설정 : n = 20, temperature = 1, top_p = 1
            scores.append(int(response.choices[0].message.content.strip()))
        return scores

In [27]:
gpt = GPT(api_filepath='../assets/openai_api_key.json')

In [42]:
# Load Documnets and summaries
ROOT = 'bitamin_auto_readme_generator'
root_absdir = os.getcwd().split(ROOT)[0]+ROOT

doc_dir = os.path.join(root_absdir,'data','object_detection','output','ocr_samples_txt')
doc_files = os.listdir(doc_dir)

summ_dir = os.path.join(root_absdir,'data','text_summarization','output','cluster_n_summary_temp1')

scores_df = pd.DataFrame(columns=['pdf','relevance','coherence','consistency','fluency'])
for filename in doc_files:
    with open(os.path.join(doc_dir, filename), 'r', encoding='utf-8') as f:
        document = f.readlines()
    if os.path.exists(os.path.join(summ_dir, filename)):
        with open(os.path.join(summ_dir, filename), 'r', encoding = 'utf-8') as f:
            summary = f.readlines()
    else:
        print("⚠️Summary doesn't exist")
        break

    # preprocess
    document = ''.join(document)
    document = erase_tag(document, 'p.\d*')
    # document = document.replace('\n','')
    summary = ''.join(summary)

    # Evaluate
    rel, coh, cons, flu =gpt.get_geval_score(document, summary, model='gpt-4o-mini')

    new_row = pd.DataFrame(data=[[filename.replace('.txt',''),rel,coh,cons,flu]],
                          columns=scores_df.columns)
    scores_df = pd.concat([scores_df, new_row], axis=0)
    

In [43]:
scores_df

,pdf,relevance,coherence,consistency,fluency
0,arima_text,4,4,5,3
0,asiancup_text,4,4,5,3
0,barbot_text,4,4,5,3
0,braintumor_text,4,4,4,2
0,hangang_text,4,4,5,3
0,interior_text,5,4,5,3
0,kospi_text,4,4,4,3
0,lier-detector_text,4,4,4,2
0,netflix_text,4,4,5,3
0,webtoon_text,4,4,5,3
